In [3]:
import PyPDF2

# Función para extraer texto de un archivo PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()  # Extraer texto de cada página
    return text

# Ruta al archivo PDF
pdf_path = "./Cenicienta.pdf"

# Extraer texto del PDF
pdf_text = extract_text_from_pdf(pdf_path)

# Separar el texto en párrafos, si lo deseas
paragraph_texts = pdf_text.split("\n")

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/root/miniconda3/envs/rag/lib/python3.13/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [6]:
from langchain_chroma import Chroma


vector_store = Chroma.from_texts(
    texts=paragraph_texts,
    collection_name="cenicienta",
    embedding=embeddings,
    persist_directory="/tmp/chroma_cuentos",
)

In [8]:
retriever = vector_store.as_retriever(
    search_type="similarity",  search_kwargs={"k": 3}
)

def search_with_retriever(query, top_k):
    try:
        retriever.search_kwargs["k"] = top_k  # Dynamically set the number of results
        results = retriever.get_relevant_documents(query)
        return "\n\n".join(
            [f"**Result {i+1}:**\n{doc.page_content}" for i, doc in enumerate(results)]
        )
    except Exception as e:
        return f"Error: {e}"

In [10]:
search_with_retriever('¿Cual es la moraleja del cuento?', 2)

/tmp/ipykernel_47761/87945450.py:8: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


'**Result 1:**\nOtra moraleja: sin duda es una gran ventaja tener inteligencia, coraje, buena crianza,y sentido\n\n**Result 2:**\ncomún. Estos y otros talentos similares solo provienen del cielo, y es bueno tenerlos. Sin'